In [14]:
import sys
sys.path.append('..')
print(sys.path)
from transformers import AutoTokenizer
import numpy as np
from sklearn.utils import resample
import pandas as pd
from nlp_pipeline.preprocess_text import TextPreprocessor
from nlp_pipeline.feature_extraction import FeatureExtractor
from model.bert.bert_fusiondataset import BertFusionDataset
from model.bert.bert_trainerfinetune import TrainerFusionBert
from model.bert.bert_finetune import FineTuneBertWithLinguistic

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

['d:\\aco\\research\\Asd-classification\\notebook', 'c:\\Users\\MufliDevs\\anaconda3\\python312.zip', 'c:\\Users\\MufliDevs\\anaconda3\\DLLs', 'c:\\Users\\MufliDevs\\anaconda3\\Lib', 'c:\\Users\\MufliDevs\\anaconda3', '', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor', '..', '..', '..', '..', '..', '..']


In [6]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
MAX_LEN = 128

def encode_texts(texts, max_len=MAX_LEN):
    encodings = tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='np'
    )
    return encodings['input_ids'], encodings['attention_mask']

In [7]:
def oversample_token_ling(input_ids, att_mask, ling_feats, labels):
    idx_min = np.where(labels == 1)[0]
    idx_maj = np.where(labels == 0)[0]

    ids_min = input_ids[idx_min]
    mask_min = att_mask[idx_min]
    ling_min = ling_feats[idx_min]
    y_min = labels[idx_min]

    ids_os, mask_os, ling_os, y_os = resample(
        ids_min, mask_min, ling_min, y_min,
        replace=True,
        n_samples=len(idx_maj),
        random_state=42
    )

    ids_final = np.concatenate([input_ids[idx_maj], ids_os], axis=0)
    mask_final = np.concatenate([att_mask[idx_maj], mask_os], axis=0)
    ling_final = np.vstack([ling_feats[idx_maj], ling_os])
    y_final = np.concatenate([labels[idx_maj], y_os])

    return ids_final, mask_final, ling_final, y_final

In [4]:
df = df = pd.read_csv('../data/feature/combined_augmented_asd.csv', encoding='latin1')
text_preprocessor = TextPreprocessor()
df['clean_text'] = df['transcription'].apply(text_preprocessor.preprocess)
df[['transcription', 'clean_text']].head()

,transcription,clean_text
0,Tunjuk kaka coba,tunjuk kaka coba
1,Inii!,ini
2,Siapa namamu?,siapa nama
3,Iyaaaa?,
4,Kenalan duluu!,kenal duluu


In [8]:
input_ids, attention_mask = encode_texts(df['clean_text'])
extractor = FeatureExtractor()
X = extractor.encode_series_bert(df['clean_text'][:len(df['label'])])
ling_cols = extractor.linguistic_cols
ling_feats = df[ling_cols].values
labels = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values

from sklearn.model_selection import train_test_split
ids_train, ids_val, mask_train, mask_val, ling_train, ling_val, y_train, y_val = train_test_split(
    input_ids, attention_mask, ling_feats, labels,
    stratify=labels, test_size=0.2, random_state=42
)

ids_train_os, mask_train_os, ling_train_os, y_train_os = oversample_token_ling(
    ids_train, mask_train, ling_train, y_train
)

train_dataset = BertFusionDataset(ids_train_os, mask_train_os, ling_train_os, y_train_os)
val_dataset = BertFusionDataset(ids_val, mask_val, ling_val, y_val)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [12]:
model = FineTuneBertWithLinguistic(
    model_name="indobenchmark/indobert-base-p1",
    ling_dim=10
)

trainer = TrainerFusionBert(model, device='cpu', epochs=10, patience=2)
model = trainer.train(train_dataset, val_dataset)

Epoch 1:
  Train Loss: 0.3830
  Val Loss  : 0.3197
  Val Acc   : 0.8616 | Precision: 0.8237 | Recall: 0.8767 | F1: 0.8494
Epoch 2:
  Train Loss: 0.2605
  Val Loss  : 0.3052
  Val Acc   : 0.8759 | Precision: 0.8346 | Recall: 0.8995 | F1: 0.8658
Epoch 3:
  Train Loss: 0.2202
  Val Loss  : 0.2904
  Val Acc   : 0.8723 | Precision: 0.8284 | Recall: 0.8995 | F1: 0.8625
Epoch 4:
  Train Loss: 0.1940
  Val Loss  : 0.3149
  Val Acc   : 0.8872 | Precision: 0.8566 | Recall: 0.8968 | F1: 0.8762
Epoch 5:
  Train Loss: 0.1978
  Val Loss  : 0.3109
  Val Acc   : 0.8878 | Precision: 0.8577 | Recall: 0.8968 | F1: 0.8768
Early stopping triggered.


In [15]:
test_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
_, test_metrics = trainer.evaluate(test_loader)
print("Validation Metrics:", test_metrics)

Validation Metrics: {'accuracy': 0.8878281622911695, 'precision': 0.8576923076923076, 'recall': 0.8967828418230563, 'f1': 0.8768020969855832}
